# Import Libraries

In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

# Transformation

We convert the image into a PyTorch tensor and scale pixel values to the range [0.0, 1.0] by dividing by 255.

We then normalize the tensor image with mean and standard deviation per channel (Red, Green, Blue) using the formula:

$$
\text{normalized_pixel} = \frac{(\text{pixel} - \mu)}{\sigma}
$$
with $$\mu = \sigma = 0.5$$
This scales the pixel values from $[0, 1]$ to $[-1, 1]$.

Why Normalize?
Neural networks train faster and more stably when input data is standardized (mean ≈ 0, std ≈ 1).


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Get Dataset

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

**Labels**

In [ ]:
class_names = ['plain', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Model

**Convolution Output Size Formula**

Given:
- **N** = input size (height or width)
- **K** = kernel size
- **P** = padding
- **S** = stride

The output size (per dimension) is:

$$
\text{Output size} = \left\lfloor \frac{N + 2P - K}{S} \right\rfloor + 1
$$

---



This formula calculates the spatial dimension of the output after applying a convolution. Padding (`P`) controls how much the input is extended at the borders. Using `P = (K - 1) // 2` and `S = 1` helps preserve the input size.


In [ ]:
class NeuralNet(nn.Module):
  def __init__(self):
    super().__init__()
    # Given the image shape of (3, 32, 32)
    self.conv1 = nn.Conv2d(3, 12, 5)  # -> (12, 28, 28)
    self.pool = nn.MaxPool2d(2, 2)  # -> (12, 14, 14)
    self.conv2 = nn.Conv2d(12, 24, 5)  # -> (24, 10, 10) -> then another pooling -> (24, 5, 5) -> then flatten -> (24 * 5 * 5)

    self.fc1 = nn.Linear(24*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x

# Train Model

In [ ]:
net = NeuralNet()

**SGD and Momentum (Optimizer)**

**SGD (Stochastic Gradient Descent)** updates model parameters using gradients to minimize the loss:

$$
\theta \leftarrow \theta - \eta \cdot \nabla L(\theta)
$$

- η (learning rate), e.g., `lr=0.001`

---

**Momentum** helps speed up SGD and smooth updates by remembering past gradients:

$$
v_t = \mu \cdot v_{t-1} - \eta \cdot \nabla L(\theta)
$$
$$
\theta = \theta + v_t
$$

- μ (momentum factor), e.g., `momentum=0.9`



In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
epoch_num = 20

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

In [ ]:
for epoch in range(epoch_num):
  print(f'Training Epoch number {epoch}')

  net.train() # set the model to training mode
  running_loss = 0.0

  for images, labels in tqdm(train_loader, desc='Training loop'):
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad() # Clear old gradients

    outputs = net(images)

    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f'Loss: {running_loss / len(train_loader):.4f}')

Saving **Model**

In [ ]:
torch.save(net.state_dict(), 'model.pth')

# Evaluating

In [ ]:
model = NeuralNet()
model.load_state_dict(torch.load('model.pth'))

In [ ]:
model.eval()

total = correct = 0

with torch.no_grad():
  for data in test_loader:
    images, labels = data

    outputs = model(images)

    _, predicted = torch.max(outputs, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy is: {accuracy} %')

# Test

In [ ]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
def loadImage(image_path):
  img = Image.open(image_path)
  img = new_transform(img)
  img = img.unsqueeze(0)  # Adds a batch dimension so shapes are compatible
  return img

In [ ]:
img_paths = ['plain.jpg']
images = [loadImage(img_path) for img_path in img_paths]

In [ ]:
model.eval()

with torch.no_grad():
  for img in images:
    outputs = model(img)
    _, predicted = torch.max(outputs, 1)

    print(f'Prediction: {class_names[predicted.item()]}')